In [ ]:
# --- imports and useful functions --- #
import pandas as pd
from common import load_data, DATA_FOLDER

# print the number of NaN values in each column
def print_na_info(df):
    for col in df.columns:
        print('{} : {}'.format(col, df[col].isna().sum()))


## 1. Convert Injury Dataset to have integer values for severity

In [12]:
# --- Create better injury datatset --- #
df_injuries = load_data('InjuryRecord.csv', DATA_FOLDER, verbose=False)

# Fix one-hot encoding 
df_injuries.loc[df_injuries['DM_M7'] == 1, 'DM_M1'] = 0
df_injuries.loc[df_injuries['DM_M28'] == 1, 'DM_M7'] = 0
df_injuries.loc[df_injuries['DM_M42'] == 1, 'DM_M28'] = 0

# Make ordinal injury severity feature
mapping = {'DM_M1': 1, 'DM_M7': 2, 'DM_M28': 3, 'DM_M42': 4}
injury_cols = ['DM_M1', 'DM_M7', 'DM_M28', 'DM_M42']
df_injuries['InjurySeverity'] = df_injuries[injury_cols].idxmax(axis=1)
df_injuries['InjurySeverity'] = df_injuries['InjurySeverity'].map(mapping)
df_injuries = df_injuries.drop(columns=['DM_M1', 'DM_M7', 'DM_M28', 'DM_M42'])
df_injuries = df_injuries.drop(columns=['PlayerKey', 'PlayKey', 'Surface'])

# Save to csv
print('Dataframe shape: {}'.format(df_injuries.shape))
print(df_injuries.head())
# df_injuries.to_csv('injuries_fixed.csv', index=False) # uncomment this to save new csv file


Dataframe shape: (105, 3)
    GameID BodyPart  InjurySeverity
0  39873-4     Knee               4
1  46074-7     Knee               2
2  36557-1    Ankle               4
3  46646-3    Ankle               1
4  43532-5    Ankle               4


## 2. Convert Tracking Dataset to be Per Game

### This also makes the dataset much smaller and easier to load...

NOTE: can't upload tracking data file to github bc its too big. For me, I have it saved at the following relative path:

    ./nfl-playing-surface-analytics/PlayerTrackData.csv

In [ ]:
# --- Create better tracking datatset --- #
df_players = load_data('PlayerTrackData.csv', DATA_FOLDER, verbose=False) 

# Reduce num of rows: get total distance for player, for each play
df_players = df_players.groupby('PlayKey')['dis'].sum().reset_index()
df_players = df_players.rename(columns={'dis': 'PlayDist'})

# Create new key features from PlayKey
df_players[['PlayerKey', 'GameNum', 'temp']] = df_players['PlayKey'].str.split('-', expand=True)
df_players['GameID'] = df_players['PlayerKey'] + '-' + df_players['GameNum']
df_players = df_players.drop(columns=['temp'])

# Reorder columns
df_players = df_players[['PlayerKey', 'GameID', 'PlayKey', 'PlayDist', 'GameNum']]

# Get GameDist and CumulativeDist for each player
df_players = df_players.groupby('GameID')['PlayDist'].sum().reset_index()
df_players = df_players.rename(columns={'PlayDist': 'GameDist'})

df_players[['PlayerNum', 'GameNum']] = df_players['GameID'].str.split('-', expand=True)
df_players['PlayerNum'] = df_players['PlayerNum'].astype(int)
df_players['GameNum'] = df_players['GameNum'].astype(int)

df_players = df_players[['PlayerNum', 'GameNum', 'GameID', 'GameDist']]
df_players = df_players.sort_values(by=['PlayerNum', 'GameNum']).reset_index(drop=True)

df_players['CumulativeDist'] = df_players.groupby('PlayerNum')['GameDist'].cumsum()

# Reorder columns andDrop uneeded columns
df_players = df_players.drop(columns=['PlayerNum', 'GameNum'])

# Save to csv
print('Dataframe shape: {}'.format(df_players.shape))
print(df_players.head())
# df_players.to_csv('distances.csv', index=False) # uncomment this to save new csv file


Dataframe shape: (5712, 3)
    GameID  GameDist  CumulativeDist
0  26624-1   1997.60         1997.60
1  26624-2   1922.03         3919.63
2  26624-3    443.61         4363.24
3  26624-4   1617.13         5980.37
4  26624-5   1682.31         7662.68


## 3. Merge datasets and create a new CSV file, specifically for experiment 2

In [14]:
# ----- STEP 1: Load all dataframes ----- #
df_injuries = load_data('injuries_fixed.csv', verbose=False)
df_distances = load_data('distances.csv', verbose=False)
df_playlist = load_data('PlayList.csv', DATA_FOLDER, verbose=False)

# Create new dataframe with relevant information from playlist
dropped_cols = ['PlayKey', 'PlayerDay', 'PlayType', 'PlayerGamePlay', 'Position', 'PositionGroup']
main_df = df_playlist.drop(columns=dropped_cols)
main_df = main_df.groupby('GameID').first().reset_index()

# ----- STEP 2: Merge Distance Data into Main Dataframe ----- #
main_df = pd.merge(main_df, df_distances, how='left', on='GameID')
main_df['GameDist'] = main_df['GameDist'].fillna(0)

# ----- STEP 3: Merge Injury Data into Main Dataframe ----- #
main_df = pd.merge(main_df, df_injuries, how='left', on='GameID')
main_df['BodyPart'] = main_df['BodyPart'].fillna('None')
main_df['InjurySeverity'] = main_df['InjurySeverity'].fillna(0)
main_df['InjurySeverity'] = main_df['InjurySeverity'].astype(int)
# print('Main df shape: {}'.format(main_df.shape))

# ----- STEP 4: Remove uneeded features and fill NaN ----- #
main_df = main_df.drop(columns=['BodyPart', 'Temperature'])
main_df['StadiumType'] = main_df['StadiumType'].fillna('Unknown')
main_df['Weather'] = main_df['Weather'].fillna('Unknown')

# ----- STEP 5: Fix Column values ----- #
main_df = main_df.rename(columns={'RosterPosition': 'Position'})
main_df['Position'] = main_df['Position'].replace({
    'Linebacker': 'LB',
    'Wide Receiver': 'WR',
    'Offensive Lineman': 'OL',
    'Safety': 'S',
    'Defensive Lineman': 'DL',
    'Cornerback': 'CB',
    'Running Back': 'RB',
    'Tight End': 'TE',
    'Quarterback': 'QB',
    'Kicker': 'K'
})

main_df['Weather'] = main_df['Weather'].replace({
    'N/A (Indoors)': 'Indoor',
    'N/A (Indoor)': 'Indoor',
    'N/A Indoor': 'Indoor',
    'Indoors': 'Indoor',
    'Controlled Climate': 'Indoor',

    'Partly Cloudy': 'PartlyCloudy',
    'Partly Clouidy': 'PartlyCloudy',
    'Party Cloudy': 'PartlyCloudy',
    'Sun & clouds': 'PartlyCloudy',
    'Mostly Cloudy': 'PartlyCloudy',
    'Partly sunny': 'PartlyCloudy',
    'Clear to Partly Cloudy': 'PartlyCloudy',
    'Partly cloudy': 'PartlyCloudy',
    'Partly Sunny': 'PartlyCloudy',
    'Partly clear': 'PartlyCloudy',

    'Coudy': 'Cloudy',
    'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.': 'Cloudy',
    'Cloudy, chance of rain': 'Cloudy',
    'Cloudy and cold': 'Cloudy',
    'Cloudy and Cool': 'Cloudy',
    'Cloudy, fog started developing in 2nd quarter': 'Cloudy',
    'Hazy': 'Cloudy',
    'Mostly Coudy': 'Cloudy',
    'Overcast': 'Cloudy',
    'cloudy': 'Cloudy',
    'Mostly cloudy': 'Cloudy',

    'Clear and Sunny': 'Clear',
    'Clear and sunny': 'Clear',
    'Sunny and clear': 'Clear',
    'Sunny and warm': 'Clear',
    'Sunny and cold': 'Clear',
    'Sunny Skies': 'Clear',
    'Sunny, Windy': 'Clear',
    'Sunny, highs to upper 80s': 'Clear',
    'Mostly Sunny Skies': 'Clear',
    'Mostly Sunny': 'Clear',
    'Mostly sunny': 'Clear',
    'Fair': 'Clear',
    'Clear Skies': 'Clear',
    'Clear skies': 'Clear',
    'Clear and Cool': 'Clear',
    'Clear and warm': 'Clear',
    'Clear and cold': 'Clear',
    'Clear and Cool': 'Clear',
    'Sunny': 'Clear',

    'Heavy lake effect snow': 'Snow',
    'Cloudy, light snow accumulating 1-3"': 'Snow',

    'Rain shower': 'Rain',
    'Cloudy, Rain': 'Rain',
    'Showers': 'Rain',
    'Rain likely, temps in low 40s.': 'Rain',
    '10% Chance of Rain': 'Rain',
    'Scattered Showers': 'Rain',
    'Rainy': 'Rain',
    'Cloudy, 50% change of rain': 'Rain',
    'Rain Chance 40%': 'Rain',
    '30% Chance of Rain': 'Rain',
    'Light Rain': 'Rain',

    'Cold': 'Unknown',
    'Heat Index 95': 'Unknown',
})

main_df['StadiumType'] = main_df['StadiumType'].replace({
    'Outdoors': 'Outdoor',
    'Oudoor': 'Outdoor',
    'Ourdoor': 'Outdoor',
    'Outddors': 'Outdoor',
    'Outside': 'Outdoor',
    'Outdor': 'Outdoor',
    'Domed, open': 'Outdoor',
    'Domed, Open': 'Outdoor',
    'Indoor, Open Roof': 'Outdoor',
    'Retr. Roof-Open': 'Outdoor',
    'Retr. Roof - Open': 'Outdoor',
    'Open': 'Outdoor',
    'Outdoor Retr Roof-Open': 'Outdoor',
    'Heinz Field': 'Outdoor',
    'Bowl': 'Outdoor',

    'Indoors': 'Indoor',
    'Indoor, Roof Closed': 'Indoor',
    'Domed, closed': 'Indoor',
    'Retr. Roof-Closed': 'Indoor',
    'Retr. Roof - Closed': 'Indoor',
    'Closed Dome': 'Indoor',
    'Dome, closed': 'Indoor',
    'Retr. Roof Closed': 'Indoor',
    'Domed': 'Indoor',
    'Dome': 'Indoor',

    'Retractable Roof': 'Unknown',
    'Cloudy': 'Unknown',
})

main_df.loc[main_df['Weather'] == 'Indoor', 'StadiumType'] = 'Indoor' # ensure indoor weather == indoor stadium

# Reorder columns
main_df = main_df[['GameID', 'PlayerKey', 'PlayerGame', 'Position', 'GameDist', 'CumulativeDist', 'StadiumType', 'FieldType', 'Weather', 'InjurySeverity']]

print('Num of injuries in dataframe: {}'.format(main_df.query('`InjurySeverity` > 0').shape[0]))
print('Dataframe shape: {}'.format(main_df.shape))
print(main_df.head())

# ----- Save to csv ----- #
# main_df.to_csv('exp2_data.csv', index=False) # uncomment this to save new csv file


Num of injuries in dataframe: 105
Dataframe shape: (5713, 10)
     GameID  PlayerKey  PlayerGame  ...  FieldType       Weather  InjurySeverity
0   26624-1      26624           1  ...  Synthetic         Clear               0
1  26624-10      26624          10  ...    Natural         Clear               0
2  26624-11      26624          11  ...  Synthetic  PartlyCloudy               0
3  26624-12      26624          12  ...  Synthetic        Cloudy               0
4  26624-13      26624          13  ...    Natural         Clear               0

[5 rows x 10 columns]
